In [1]:
# First, import necessary packages
import numpy as np
import pysindy as ps
from pysindy.feature_library import CustomLibrary
from pysindy.optimizers import STLSQ, SR3
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.preprocessing import RobustScaler

# Load your data
try:
    X = np.load('../data/block_slider_data.npy')
    t = np.load('../data/block_slider_time.npy')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Data files not found. Check paths.")

# Define physical constants for friction terms
f0 = 0.6        # Baseline friction coefficient
a = 0.015       # Direct velocity strengthening
b = 0.02        # State-based weakening 
Dc = 0.2        # Critical distance
V0 = 1e-6       # Reference velocity
k = 500000      # Spring constant
Vp = 1e-9       # Plate velocity

# Extract variables
x_data = X[:, 0]       # Position
v_data = X[:, 1]       # Velocity (slip rate)
state_data = X[:, 2]   # State variable
tau_data = X[:, 3]     # Shear stress

# Preprocess the data - only apply minimal preprocessing to avoid numerical issues
window_size = 21  # For velocity smoothing
poly_order = 2
v_smooth = savgol_filter(v_data, window_length=window_size, polyorder=poly_order)

# Create a copy with smoothed velocity
X_smooth = X.copy()
X_smooth[:, 1] = np.maximum(v_smooth, 1e-12)  # Ensure positive velocity

# Scale the data - using log scaling for velocity which spans many orders of magnitude
X_log = np.zeros_like(X_smooth)
for i in range(X_smooth.shape[1]):
    if i == 1:  # Velocity
        X_log[:, i] = np.log10(np.maximum(X_smooth[:, i], 1e-12))
    else:
        X_log[:, i] = X_smooth[:, i]

# Apply robust scaling
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_log)

# Define custom functions for the rate-state friction terms
def identity_func(x):
    """Identity function for basic terms"""
    return x

def exp_state_term(x):
    """Exponential term in the state evolution equation"""
    # Extract the state variable and scale it back
    state_scaled = x[:, 2]
    state = scaler.inverse_transform(np.column_stack([
        np.zeros_like(state_scaled),
        np.zeros_like(state_scaled),
        state_scaled,
        np.zeros_like(state_scaled)
    ]))[:, 2]
    
    return np.exp((f0 - state)/b)

def velocity_ratio(x):
    """v/V0 term in state evolution equation"""
    # Extract velocity and scale it back
    v_scaled = x[:, 1]
    v_log = scaler.inverse_transform(np.column_stack([
        np.zeros_like(v_scaled),
        v_scaled,
        np.zeros_like(v_scaled),
        np.zeros_like(v_scaled)
    ]))[:, 1]
    
    v = 10**v_log  # Convert back from log scale
    return v/V0

def const_term(x):
    """Constant term (1)"""
    return np.ones(x.shape[0])

# Create arrays of functions and names
functions = [
    const_term,
    lambda x: x[:, 1],  # Velocity
    exp_state_term,
    velocity_ratio,
    lambda x: x[:, 0],  # Position
    lambda x: x[:, 2],  # State
    lambda x: x[:, 3],  # Stress
    lambda x: np.sign(x[:, 1])  # Sign of velocity
]

function_names = [
    "1",
    "v",
    "exp((f0-state)/b)",
    "v/V0",
    "x",
    "state",
    "tau",
    "sign(v)"
]

# Create the custom library
custom_lib = CustomLibrary(
    library_functions=functions,
    function_names=function_names,
    include_bias=False  # We already include a constant term
)

# Set up SINDy with the custom library
print("Fitting SINDy with custom rate-state friction library...")
optimizer = STLSQ(threshold=1e-3)
model = ps.SINDy(
    optimizer=optimizer,
    feature_library=custom_lib,
    feature_names=["x", "v", "state", "tau"]
)

# Compute time step for derivatives
dt = np.median(np.diff(t))
print(f"Using time step: {dt}")

# Fit the model
try:
    model.fit(X_scaled, t=dt)
    print("Model fitted successfully!")
    
    # Print the discovered equations
    print("\nDiscovered equations:")
    model.print()
    
    # Print the coefficients
    print("\nModel coefficients:")
    coef_matrix = model.coefficients()
    
    # Create a heatmap of coefficients
    plt.figure(figsize=(10, 8))
    plt.imshow(np.abs(coef_matrix) > 1e-6, cmap='binary', aspect='auto')
    plt.colorbar(label='Non-zero')
    plt.xlabel('Equation')
    plt.ylabel('Feature')
    plt.title('Sparsity Pattern of Discovered Equations')
    plt.xticks(range(4), ["x", "v", "state", "tau"])
    plt.yticks(range(len(function_names)), function_names)
    plt.savefig('../figures/custom_library_sparsity.png')
    plt.show()
    
except Exception as e:
    print(f"Error fitting model: {e}")
    print("try a different approach.")

Data loaded successfully.
Fitting SINDy with custom rate-state friction library...
Using time step: 31536000.0
Error fitting model: too many indices for array: array is 1-dimensional, but 2 were indexed
try a different approach.
